In [1]:
import scipy.io
import numpy as np
import pandas as pd
import os
import itertools
import shutil
import csv

In [2]:
ann = scipy.io.loadmat('../../data/Annotations.mat')

In [3]:
ann.keys()

dict_keys(['val', '__header__', '__version__', 'test', 'train', '__globals__'])

In [4]:
annt = np.transpose(ann['test'])

In [5]:
ann_train = np.transpose(ann['train'])
ann_train.shape

(17077, 1)

In [6]:
ann_val = np.transpose(ann['val'])
ann_val.shape

(2088, 1)

In [7]:
emodb_small = []
for elem in annt:
    e = elem[0]
    if e[1]=='emodb_small/images':
        emodb_small.append(e)

In [8]:
len(emodb_small)

399

In [9]:
for elem in ann_train:
    e = elem[0]
    if e[1]=='emodb_small/images':
        emodb_small.append(e)
        
len(emodb_small)

1211

In [10]:
for elem in ann_val:
    e = elem[0]
    if e[1]=='emodb_small/images':
        emodb_small.append(e)
        
len(emodb_small)

1374

In [13]:
names = os.listdir('../../data/TestImgs_48/')
len(names)

256

In [14]:
imgs48 = []
for elem in emodb_small:
    if elem[0][0] in names:
        imgs48.append(elem)

In [15]:
len(imgs48)

256

In [16]:
for i in imgs48[1][4][0][0]:
    print(i)
    print("---------------------------------------------")

[[ 66.  17. 319. 239.]]
---------------------------------------------
[[(array([[array(['Anticipation'], dtype='<U12'),
        array(['Confidence'], dtype='<U10'),
        array(['Engagement'], dtype='<U10'),
        array(['Disapproval'], dtype='<U11'),
        array(['Doubt/Confusion'], dtype='<U15')]], dtype=object),)
  (array([[array(['Surprise'], dtype='<U8'),
        array(['Annoyance'], dtype='<U9')]], dtype=object),)
  (array([[array(['Anticipation'], dtype='<U12'),
        array(['Excitement'], dtype='<U10'),
        array(['Confidence'], dtype='<U10'),
        array(['Annoyance'], dtype='<U9')]], dtype=object),)]]
---------------------------------------------
[[array(['Annoyance'], dtype='<U9') array(['Anticipation'], dtype='<U12')
  array(['Confidence'], dtype='<U10')
  array(['Disapproval'], dtype='<U11')
  array(['Doubt/Confusion'], dtype='<U15')
  array(['Engagement'], dtype='<U10') array(['Excitement'], dtype='<U10')
  array(['Surprise'], dtype='<U8')]]
----------------

In [17]:
emotions = []
for elem in imgs48:
    #print(elem[0][0])
    e = elem[4].tolist()
    e1 = e[0]
    for item in e1:
        li = item[1].tolist()
        l2 = li[0]
        emotions_item = []
        for thing in l2:
            for t in thing:
                t1 = t.tolist()
                t2 = t1[0]
                arr = np.vstack(t2).tolist()
                arr2 = list(itertools.chain.from_iterable(arr))
                emotions_item.append(arr2)
    labels = list(itertools.chain.from_iterable(emotions_item))
    emotions.append([elem[0], labels])

emotions = np.array(emotions)
#print(emotions)

In [18]:
for i in emotions[5]:
    print(i)
    print('----------------------------------')

['csnkvsyn0xwl7kjgw4.jpg']
----------------------------------
['Disapproval', 'Doubt/Confusion', 'Annoyance', 'Disconnection', 'Disquietment', 'Annoyance', 'Annoyance', 'Disapproval']
----------------------------------


In [19]:
#print(len(emotions)) #256
print(emotions[0][1])

['Anger', 'Aversion', 'Annoyance', 'Disapproval', 'Confidence', 'Engagement']


In [20]:
unique_emo = []
for temp in emotions:
    if len(unique_emo)!=0:
        for emotion in temp[1]:
            if emotion in unique_emo:
                continue
            else:
                unique_emo.append(emotion)
    else:
        unique_emo.append(emotion for emotion in temp[1])

In [21]:
unique_emo

[<generator object <genexpr> at 0x7fd8d2bc0678>,
 'Anticipation',
 'Confidence',
 'Engagement',
 'Disapproval',
 'Doubt/Confusion',
 'Surprise',
 'Annoyance',
 'Excitement',
 'Pleasure',
 'Happiness',
 'Affection',
 'Sympathy',
 'Suffering',
 'Sadness',
 'Sensitivity',
 'Pain',
 'Fear',
 'Aversion',
 'Anger',
 'Disconnection',
 'Disquietment',
 'Peace',
 'Esteem',
 'Fatigue',
 'Yearning',
 'Embarrassment']

In [22]:
#7 classes for trained classifier:
train_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'] 
converted_classes = ['Anger', 'Aversion', 'Fear', 'Happiness','Sadness','Surprise']
def convert(emotion):
    for index,label in enumerate(converted_classes):
        if emotion == label:
            return train_classes[index]
        

In [23]:
emotions[0][1]

['Anger', 'Aversion', 'Annoyance', 'Disapproval', 'Confidence', 'Engagement']

In [24]:
update_emotions = []
update_imgs = []
for emo in emotions:
    for index, emotion in enumerate(emo[1]):         
        if emotion in converted_classes:
            update_emotions.append(convert(emotion))
            update_imgs.append(emo[0][0])
            break
        else:
            if index == (len(emo[1])-1):
                update_emotions.append('Neutral')
                update_imgs.append(emo[0][0])
            else:
                continue
    
    

In [26]:
len(update_imgs)

256

In [27]:
'''
dst = '../../data/imgs/'
src = '../../data/TestImgs_48/'
files = os.listdir(src)
for f in files:
    if f in update_imgs:
        shutil.move(src+f, dst+f)
    else:
        continue
'''

"\ndst = '../../data/imgs/'\nsrc = '../../data/TestImgs_48/'\nfiles = os.listdir(src)\nfor f in files:\n    if f in update_imgs:\n        shutil.move(src+f, dst+f)\n    else:\n        continue\n"

In [28]:
with open('../../data/label.csv','w') as searized_label:
    wr = csv.writer(searized_label,dialect='excel')
    wr.writerow(update_emotions)

In [29]:
pd_labels = pd.read_csv('../../data/label.csv',delimiter=',',header=None)
pd_labels

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,Angry,Surprise,Happy,Sad,Angry,Neutral,Sad,Angry,Happy,Surprise,...,Angry,Happy,Angry,Surprise,Angry,Happy,Happy,Sad,Angry,Sad
